# Introdução Machine Learning - Data ICMC-USP

## Tarefa Aula 01 - k-Nearest Neighbors

Vamos começar carregando os dados que iremos usar no nossa tarefa. Esses dados fornecem várias informações a respeito de diferentes vinhos e o objetivo é classificar se o vinho é bom (target é a coluna *is_good*).

Esse conjunto de dados é uma modificação do conjunto 

In [1]:
import pandas as pd

df = pd.read_csv('wine.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is good
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0


In [2]:
shape = df.shape
print(shape)

(1599, 12)


In [3]:
for col in df.columns:
    print(col)

fixed acidity
volatile acidity
citric acid
residual sugar
chlorides
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol
is good


### Deixando os dados na mesma escala
Para vários algoritmos é importante deixarmos os dados em uma mesma escala, e o kNN um desses casos. Para entender melhor vamos olhar o exemplo a seguir:


Nesse caso a distância entre os dois pontos é dada por

$$
\begin{align*}
\text{dist}(x^{(1)}, x^{(2)}) &= \sqrt{(x^{(1)}_1 - x^{(2)}_1)^2 + (x^{(1)}_2 - x^{(2)}_2)^2} \\
  &= \sqrt{(3 - 2)^2 + (10000 - 9000)^2} \\
  &= \sqrt{1 + 1000000} \\
  &= \sqrt{1000001} \\
  &= 1000.0005
\end{align*}$$


Como as escalas são muito diferentes o primeiro atributo acaba não interferindo em praticamente nada no resultado da distância. E é importante perceber que esse tipo de situação ocorre com frequência em conjuntos de dados reais.

Existem diversas formas de tratar essa situação, aqui usaremos uma técnica chamada **Min-Max Scaling**, que transforma os dados deixando-os no intervalo $[0, 1]$. A formula é da transformação é a seguinte:

$$x^{(i)}_j \leftarrow \frac{x^{(i)}_j - min(x_j)}{max(x_j) - min(x_j)}$$

Em palavras significa que vamos subtrair o menor valor da atributo e dividir pela amplitude (diferença entre o máximo e o mínimo).


Pronto, agora que entendemos podemos fazer fazer isso para todas as nossas colunas utilizando a função interna do scikit-learn

In [4]:
X = df.drop('is good', axis=1)
Y = df['is good']

from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)

### Divisão dos dados em treino e validação

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
x_test.shape

(320, 11)

### Treinando um modelo

In [7]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(x_train,y_train)

from knn import knn_predictions
from knn import knn_accuracy

y_pred = clf.predict(x_test)

### Avaliando o modelo treinado

In [8]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
print(f'A acurácia foi de {acc * 100:.2f}%')

A acurácia foi de 71.88%


### Explorando variações no modelo

#### Número de vizinhos

O principal hiperparâmetro do kNN é justamente o número de vizinhos, representado pelo k. Por padrão o `KNeighborsClassifier()` usa cinco vizinhos, através de seu parâmetro `n_neighbors` é possível alterar este valor.

#### Métrica de distância

Como vimos na aula, é possível utilizar diferentes metricas de distancia entre pontos, e vimos as duas seguintes:

- Distância Euclidiana => $dist(a, b) = \sqrt{\sum_i (a_i - b_i)^2}$
- Distância Manhattan => $dist(a, b) = \sum_i |a_i - b_i|$

O sklearn, por outro lado, faz uso de uma generalização destas duas distâncias, chamada distância **Minkowski** =>
$dist(a, b) = (\sum_i |a_i - b_i|^p)^\frac{1}{p}$. Perceba que com $p=2$ temos a distância Euclidiano e com $p=1$ temos a distância Manhattan. 

Por padrão a classe `KNeighborsClassifier()` usa `p=2`.

In [10]:
n_vizinhos = [3, 5, 7, 9, 11, 13]
resultados = []

# Utilizando distância de Manhattan obtemos uma acurácia melhor

for k in n_vizinhos:
    
    from sklearn.neighbors import KNeighborsClassifier
    from knn import knn_predictions
    from knn import knn_accuracy
    
    knn = KNeighborsClassifier(n_neighbors = k, p = 2)
    knn.fit(x_train,y_train)
    pred = knn.predict(x_test)
    acc = accuracy_score(y_test, pred)
    resultados.append(acc)

for k, acc in zip(n_vizinhos, resultados):
    print(f'{k:02d} vizinhos => Acurácia {acc * 100:.2f}%')

03 vizinhos => Acurácia 71.88%
05 vizinhos => Acurácia 68.44%
07 vizinhos => Acurácia 70.94%
09 vizinhos => Acurácia 70.00%
11 vizinhos => Acurácia 72.81%
13 vizinhos => Acurácia 71.25%
